In [1]:
from huggingface_hub import login
with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [2]:
from accelerate import Accelerator
from accelerate.utils import gather_object
from transformers import AutoModelForCausalLM, AutoTokenizer
from statistics import mean
import torch, time, json

accelerator = Accelerator()

# 10*10 Prompts. Source: https://www.penguin.co.uk/articles/2022/04/best-first-lines-in-books
prompts_all=[
    "The King is dead. Long live the Queen.",
    "Once there were four children whose names were Peter, Susan, Edmund, and Lucy.",
    "The story so far: in the beginning, the universe was created.",
    "It was a bright cold day in April, and the clocks were striking thirteen.",
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",
    "The sweat wis lashing oafay Sick Boy; he wis trembling.",
    "124 was spiteful. Full of Baby's venom.",
    "As Gregor Samsa awoke one morning from uneasy dreams he found himself transformed in his bed into a gigantic insect.",
    "I write this sitting in the kitchen sink.",
    "We were somewhere around Barstow on the edge of the desert when the drugs began to take hold.",
] * 10

# load a base model and tokenizer
model_path = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_path,    
    device_map={"": accelerator.process_index},
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)   

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [3]:
prompts_all= [
    {"role": "system", "content": "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets. \
        Compute common financial ratios and then determine the buy sell decision. Explain your reasons and answer in a format that compiles to a JSON object.\
        Answer as a JSON string with the following example format: \
        {'Investment Decision': BUY, 'Reason': 'Gross profit and EPS have both increased over time'}"},
    {"role": "user", "content": """                                                          t           t-1           t-2           t-3           t-4           t-5
level_1                                                                                                                           
01 Revenue (Adj)                                2.960000e+09  3.357800e+09  2.739700e+09  2.662700e+09  1.897400e+09  2.366300e+09
05 Cost of Revenue (Adj)                        1.743700e+09  2.212400e+09  1.695000e+09  1.692700e+09  9.701000e+08  1.477500e+09
08 Gross Profit (Adj)                           1.216300e+09  1.145400e+09  1.044700e+09  9.700000e+08  9.273000e+08  8.888000e+08
10 Operating Expenses (Adj)                     1.010700e+09  9.740000e+08  8.744000e+08  7.806000e+08  7.339000e+08  6.857000e+08
14 Operating Income or Losses (Adj)             2.056000e+08  1.714000e+08  1.703000e+08  1.894000e+08  1.934000e+08  2.031000e+08
17 Interest Expense (Adj)                       4.910000e+07  4.680000e+07  4.120000e+07  3.730000e+07  3.850000e+07  4.600000e+07
18 Interest Income (Adj)                        9.000000e+06  6.000000e+06  2.300000e+06  0.000000e+00  0.000000e+00  0.000000e+00
21 Pretax Income (Loss), Adjusted (Adj)         1.741000e+08  1.710000e+08  1.053000e+08  1.694000e+08  1.856000e+08  1.812000e+08
22 Abnormal Losses (Gains)                      3.100000e+06 -4.730000e+07  3.260000e+07 -2.370000e+07  0.000000e+00  0.000000e+00
28 Pretax Income (Loss), GAAP                   1.741000e+08  1.710000e+08  1.053000e+08  1.694000e+08  1.856000e+08  1.812000e+08
29 Income Tax Expense (Benefit)                 4.580000e+07  3.470000e+07  3.020000e+07  5.960000e+07  5.760000e+07  5.910000e+07
32 Income (Loss) from Continuing Operations     1.283000e+08  1.363000e+08  7.510000e+07  1.098000e+08  1.280000e+08  1.221000e+08
33 Net Extraordinary Losses (Gains)             0.000000e+00  0.000000e+00  0.000000e+00  4.500000e+06  0.000000e+00  0.000000e+00
36 Net Income Including Minority Interest       1.283000e+08  1.363000e+08  7.510000e+07  1.053000e+08  1.280000e+08  1.221000e+08
37 Net Income/Net Profit (Losses)               1.283000e+08  1.363000e+08  7.510000e+07  1.053000e+08  1.280000e+08  1.221000e+08
38 Preferred Dividends                          0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+05  2.000000e+05  3.000000e+05
40 Net Income Avail to Common, GAAP             1.283000e+08  1.363000e+08  7.510000e+07  1.052000e+08  1.278000e+08  1.218000e+08
41 Net Income Avail to Common, Adj (Adj)        1.283000e+08  1.363000e+08  7.510000e+07  1.052000e+08  1.278000e+08  1.218000e+08
43 Net Extraordinary Losses (Gains)             0.000000e+00  0.000000e+00  0.000000e+00  4.500000e+06  0.000000e+00  0.000000e+00
44 Basic Weighted Average Number of Shares      4.460000e+07  4.420000e+07  4.410000e+07  4.400000e+07  4.410000e+07  4.510000e+07
45 Basic Earnings per Share                     2.880000e+00  3.080000e+00  1.710000e+00  2.390000e+00  2.900000e+00  2.700000e+00
46 Basic EPS from Continuing Operations         2.880000e+00  3.080000e+00  1.710000e+00  2.490000e+00  2.900000e+00  2.700000e+00
47 Basic EPS from Continuing Operations         3.000000e+00  2.290000e+00  2.180000e+00  2.140000e+00  2.900000e+00  2.700000e+00
48 Diluted Weighted Average Shares              4.470000e+07  4.440000e+07  4.430000e+07  4.420000e+07  4.430000e+07  4.520000e+07
49 Diluted EPS                                  2.870000e+00  3.070000e+00  1.700000e+00  2.380000e+00  2.880000e+00  2.690000e+00
50 Diluted EPS from Continuing Operations       2.870000e+00  3.070000e+00  1.700000e+00  2.480000e+00  2.880000e+00  2.690000e+00
51 Diluted EPS from Continuing Operations, Adj  2.990000e+00  2.280000e+00  2.170000e+00  2.130000e+00  2.880000e+00  2.690000e+0"""}

]

In [12]:
prompt = "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets.         Compute common financial ratios and then determine the buy sell decision. Explain your reasons and answer in a format that compiles to a JSON object.        Answer as a JSON string with the following example format:         {'Investment Decision': BUY, 'Reason': 'Gross profit and EPS have both increased over time'}                                                          t           t-1           t-2           t-3           t-4           t-5\nlevel_1                                                                                                                           \n01 Revenue (Adj)                                2.960000e+09  3.357800e+09  2.739700e+09  2.662700e+09  1.897400e+09  2.366300e+09\n05 Cost of Revenue (Adj)                        1.743700e+09  2.212400e+09  1.695000e+09  1.692700e+09  9.701000e+08  1.477500e+09\n08 Gross Profit (Adj)                           1.216300e+09  1.145400e+09  1.044700e+09  9.700000e+08  9.273000e+08  8.888000e+08\n10 Operating Expenses (Adj)                     1.010700e+09  9.740000e+08  8.744000e+08  7.806000e+08  7.339000e+08  6.857000e+08\n14 Operating Income or Losses (Adj)             2.056000e+08  1.714000e+08  1.703000e+08  1.894000e+08  1.934000e+08  2.031000e+08\n17 Interest Expense (Adj)                       4.910000e+07  4.680000e+07  4.120000e+07  3.730000e+07  3.850000e+07  4.600000e+07\n18 Interest Income (Adj)                        9.000000e+06  6.000000e+06  2.300000e+06  0.000000e+00  0.000000e+00  0.000000e+00\n21 Pretax Income (Loss), Adjusted (Adj)         1.741000e+08  1.710000e+08  1.053000e+08  1.694000e+08  1.856000e+08  1.812000e+08\n22 Abnormal Losses (Gains)                      3.100000e+06 -4.730000e+07  3.260000e+07 -2.370000e+07  0.000000e+00  0.000000e+00\n28 Pretax Income (Loss), GAAP                   1.741000e+08  1.710000e+08  1.053000e+08  1.694000e+08  1.856000e+08  1.812000e+08\n29 Income Tax Expense (Benefit)                 4.580000e+07  3.470000e+07  3.020000e+07  5.960000e+07  5.760000e+07  5.910000e+07\n32 Income (Loss) from Continuing Operations     1.283000e+08  1.363000e+08  7.510000e+07  1.098000e+08  1.280000e+08  1.221000e+08\n33 Net Extraordinary Losses (Gains)             0.000000e+00  0.000000e+00  0.000000e+00  4.500000e+06  0.000000e+00  0.000000e+00\n36 Net Income Including Minority Interest       1.283000e+08  1.363000e+08  7.510000e+07  1.053000e+08  1.280000e+08  1.221000e+08\n37 Net Income/Net Profit (Losses)               1.283000e+08  1.363000e+08  7.510000e+07  1.053000e+08  1.280000e+08  1.221000e+08\n38 Preferred Dividends                          0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+05  2.000000e+05  3.000000e+05\n40 Net Income Avail to Common, GAAP             1.283000e+08  1.363000e+08  7.510000e+07  1.052000e+08  1.278000e+08  1.218000e+08\n41 Net Income Avail to Common, Adj (Adj)        1.283000e+08  1.363000e+08  7.510000e+07  1.052000e+08  1.278000e+08  1.218000e+08\n43 Net Extraordinary Losses (Gains)             0.000000e+00  0.000000e+00  0.000000e+00  4.500000e+06  0.000000e+00  0.000000e+00\n44 Basic Weighted Average Number of Shares      4.460000e+07  4.420000e+07  4.410000e+07  4.400000e+07  4.410000e+07  4.510000e+07\n45 Basic Earnings per Share                     2.880000e+00  3.080000e+00  1.710000e+00  2.390000e+00  2.900000e+00  2.700000e+00\n46 Basic EPS from Continuing Operations         2.880000e+00  3.080000e+00  1.710000e+00  2.490000e+00  2.900000e+00  2.700000e+00\n47 Basic EPS from Continuing Operations         3.000000e+00  2.290000e+00  2.180000e+00  2.140000e+00  2.900000e+00  2.700000e+00\n48 Diluted Weighted Average Shares              4.470000e+07  4.440000e+07  4.430000e+07  4.420000e+07  4.430000e+07  4.520000e+07\n49 Diluted EPS                                  2.870000e+00  3.070000e+00  1.700000e+00  2.380000e+00  2.880000e+00  2.690000e+00\n50 Diluted EPS from Continuing Operations       2.870000e+00  3.070000e+00  1.700000e+00  2.480000e+00  2.880000e+00  2.690000e+00\n51 Diluted EPS from Continuing Operations, Adj  2.990000e+00  2.280000e+00  2.170000e+00  2.130000e+00  2.880000e+00  2.690000e+0"

In [4]:
prompts_all

[{'role': 'system',
  'content': "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets.         Compute common financial ratios and then determine the buy sell decision. Explain your reasons and answer in a format that compiles to a JSON object.        Answer as a JSON string with the following example format:         {'Investment Decision': BUY, 'Reason': 'Gross profit and EPS have both increased over time'}"},
 {'role': 'user',
  'content': '                                                          t           t-1           t-2           t-3           t-4           t-5\nlevel_1                                                                                                                           \n01 Revenue (Adj)                                2.960000e+09  3.357800e+09  2.739700e+09  2.662700e+09  1.897400e+09  2.366300e+09\n05 Cost of Revenue (Adj)                        1.743700e+09  2.212400e+09  1.695000e+09  

In [24]:
# sync GPUs and start the timer
accelerator.wait_for_everyone()
start=time.time()

# divide the prompt list onto the available GPUs 
with accelerator.split_between_processes(prompts_all) as prompts:
    # store output of generations in dict
    results=dict(outputs=[], num_tokens=0)

    # have each GPU do inference, prompt by prompt
    for prompt in prompts:
        prompt_tokenized=tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True, return_tensors='pt').to("cuda")#(prompt, return_tensors="pt").to("cuda")
        output_tokenized = model.generate(**prompt_tokenized, max_new_tokens=200)[0]

        # remove prompt from output 
        output_tokenized=output_tokenized[len(prompt_tokenized["input_ids"][0]):]

        # store outputs and number of tokens in result{}
        results["outputs"].append( tokenizer.decode(output_tokenized) )
        results["num_tokens"] += len(output_tokenized)

    results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

# collect results from all the GPUs
results_gathered=gather_object(results)

if accelerator.is_main_process:
    timediff=time.time()-start
    num_tokens=sum([r["num_tokens"] for r in results_gathered ])

    print(f"tokens/sec: {num_tokens//timediff}, time {timediff}, total tokens {num_tokens}, total prompts {len(prompts_all)}")

UndefinedError: dict object has no element 0

In [17]:
results_gathered[0]['outputs']

[" I am trying to remember the date of the last time I saw my friend and I wanted to know if you could help me.\n\nI am trying to remember the date of the last time I saw my friend. You know, the one we met in the mall last year. I'm pretty sure it was around the time the new iPhone was released. I remember we went to the Apple store together to see the new model and then we went to dinner at that new Italian place that just opened up downtown.\n\nI'm trying to figure out what date it was. Let me think for a moment... Let's see, I think it was around the 15th of June. Is that correct? Or was it the 20th? I'm really sorry, I'm getting a little mixed up. I know it was around that time, but I'm not sure which day.\n\nPlease let me know if you could help me figure out the date of the last time I saw my friend. I would"]

In [2]:
from accelerate import Accelerator
from accelerate.utils import gather_object
from transformers import AutoModelForCausalLM, AutoTokenizer
from statistics import mean
import torch, time, json

def write_pretty_json(file_path, data):
    import json
    with open(file_path, "w") as write_file:
        json.dump(data, write_file, indent=4)

# 10*10 Prompts. Source: https://www.penguin.co.uk/articles/2022/04/best-first-lines-in-books
prompts_all=[
    "The King is dead. Long live the Queen.",
    "Once there were four children whose names were Peter, Susan, Edmund, and Lucy.",
    "The story so far: in the beginning, the universe was created.",
    "It was a bright cold day in April, and the clocks were striking thirteen.",
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",
    "The sweat wis lashing oafay Sick Boy; he wis trembling.",
    "124 was spiteful. Full of Baby's venom.",
    "As Gregor Samsa awoke one morning from uneasy dreams he found himself transformed in his bed into a gigantic insect.",
    "I write this sitting in the kitchen sink.",
    "We were somewhere around Barstow on the edge of the desert when the drugs began to take hold.",
] * 10

# load a base model and tokenizer
model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)   
tokenizer.pad_token = tokenizer.eos_token

# batch, left pad (for inference), and tokenize
def prepare_prompts(prompts, tokenizer, batch_size=16):
    batches=[prompts[i:i + batch_size] for i in range(0, len(prompts), batch_size)]  
    batches_tok=[]
    tokenizer.padding_side="left"     
    for prompt_batch in batches:
        batches_tok.append(
            tokenizer(
                prompt_batch, 
                return_tensors="pt", 
                padding='longest', 
                truncation=False, 
                pad_to_multiple_of=8,
                add_special_tokens=False).to("cuda") 
            )
    tokenizer.padding_side="right"
    return batches_tok

def run_inference():
    # sync GPUs and start the timer
    accelerator = Accelerator()
    model = AutoModelForCausalLM.from_pretrained(
        model_path,    
        device_map={"": accelerator.process_index},
        torch_dtype=torch.bfloat16,
    )
    accelerator.wait_for_everyone()    
    start=time.time()

    # divide the prompt list onto the available GPUs 
    with accelerator.split_between_processes(prompts_all) as prompts:
        results=dict(outputs=[], num_tokens=0)

        # have each GPU do inference in batches
        prompt_batches=prepare_prompts(prompts, tokenizer, batch_size=16)

        for prompts_tokenized in prompt_batches:
            outputs_tokenized=model.generate(**prompts_tokenized, max_new_tokens=100)

            # remove prompt from gen. tokens
            outputs_tokenized=[ tok_out[len(tok_in):] 
                for tok_in, tok_out in zip(prompts_tokenized["input_ids"], outputs_tokenized) ] 

            # count and decode gen. tokens 
            num_tokens=sum([ len(t) for t in outputs_tokenized ])
            outputs=tokenizer.batch_decode(outputs_tokenized)

            # store in results{} to be gathered by accelerate
            results["outputs"].extend(outputs)
            results["num_tokens"] += num_tokens

        results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

    # collect results from all the GPUs
    results_gathered=gather_object(results)

    timediff=time.time()-start
    num_tokens=sum([r["num_tokens"] for r in results_gathered ])

    print(f"tokens/sec: {num_tokens//timediff}, time elapsed: {timediff}, num_tokens {num_tokens}")

In [1]:
from accelerate import notebook_launcher

In [3]:
notebook_launcher(run_inference, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tokens/sec: 2593.0, time elapsed: 3.8558549880981445, num_tokens 10000tokens/sec: 2593.0, time elapsed: 3.8559179306030273, num_tokens 10000tokens/sec: 2593.0, time elapsed: 3.8559176921844482, num_tokens 10000tokens/sec: 2593.0, time elapsed: 3.8559558391571045, num_tokens 10000





[2025-02-17 22:13:32,510] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 873 via signal SIGTERM
[2025-02-17 22:13:32,511] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 874 via signal SIGTERM
[2025-02-17 22:13:32,512] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 875 via signal SIGTERM


In [4]:
torch.cuda.empty_cache()

In [7]:
torch.cuda.device_count()

2

In [10]:
torch.cuda.memory_reserved()/1024**2

736.0

In [11]:
torch.cuda.memory_cached()/1024**2

/opt/kernel/lib/python3.11/site-packages/torch/cuda/memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


736.0

In [1]:
import torch
import torch.nn as nn


def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_cached()/1024**2)


def allocate():
    x = torch.randn(1024*1024, device='cuda')
    memory_stats()
    
    
memory_stats()
# 0.0
# 0.0

allocate()
# 4.0 # allocated inside the function
# 20.0 # used cache

memory_stats()
# 0.0 # local tensor is free
# 20.0 # cache is still alive

torch.cuda.empty_cache()
memory_stats()
# 0.0
# 0.0 # cache is free again

x = torch.randn(1024, 1024, device='cuda')
memory_stats()
# 4.0
# 20.0

# store referece
y = x

del x # this does not free the memory of x since y still points to it
memory_stats() 
# 4.0  
# 20.0

del y # this allows PyTorch to free the memory and reuse it in the cache
memory_stats()
# 0.0
# 20.0

torch.cuda.empty_cache()
memory_stats() 
# 0.0
# 0.0

0.0
0.0


/opt/kernel/lib/python3.11/site-packages/torch/cuda/memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


4.0
20.0
0.0
20.0
0.0
0.0
4.0
20.0
4.0
20.0
0.0
20.0
0.0
0.0


In [2]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [6]:
del model

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from accelerate import PartialState, prepare_pippy

In [8]:
torch.__version__

'2.4.0.post301'

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct", low_cpu_mem_usage=True, attn_implementation="sdpa"
)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [10]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
prompts = ("I would like to", "I really like to")  # bs = 2, sending 2 per process
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompts, return_tensors="pt", padding=True)

In [36]:
%package install pytorch==2.4.0 

Running: micromamba install pytorch==2.4.0 --yes --quiet --log-level=error

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [1]:
#model = prepare_pippy(model, split_points="auto", example_kwargs=inputs)

In [12]:
rank = int(os.environ["RANK"])
device = torch.device(f"cuda:{rank}")
torch.distributed.init_process_group("nccl", device_id=device)

KeyError: 'RANK'

In [3]:
torch.device(5)

device(type='cuda', index=5)

In [5]:
import torch
from accelerate import Accelerator
from accelerate.utils import gather_object
from transformers import AutoTokenizer, AutoModelForCausalLM

accelerator = Accelerator()

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    device_map={"": accelerator.process_index},
    torch_dtype=torch.float16
)

In [6]:
prompts_all = [
    "Hello what is your name?",
    "The weather today is"
]

# sync GPUs and start the timer
accelerator.wait_for_everyone()

# divide the prompt list onto the available GPUs 
with accelerator.split_between_processes(prompts_all) as prompts:
    # store output of generations in dict
    results=dict(outputs=[], num_tokens=0)

    # have each GPU do inference, prompt by prompt
    for prompt in prompts:
        prompt_tokenized=tokenizer(prompt, return_tensors="pt").to("cuda")
        output_tokenized = model.generate(**prompt_tokenized, max_new_tokens=100)[0]

        # remove prompt from output 
        output_tokenized=output_tokenized[len(prompt_tokenized["input_ids"][0]):]

        # store outputs and number of tokens in result{}
        results["outputs"].append( tokenizer.decode(output_tokenized) )
        results["num_tokens"] += len(output_tokenized)

    results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

# collect results from all the GPUs
results_gathered=gather_object(results)

print(results_gathered)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'outputs': [" I'm a huge fan of your work. I was wondering if you would be willing to share your contact information so that I can get in touch with you. I would love to learn more about your projects and potentially collaborate on future endeavors.\n\nThank you so much for your time and consideration. I look forward to hearing from you soon!\n\nBest regards,\n[Your Name]<|eot_id|>", " lovely, isn't it?\nI was thinking of taking a walk outside and enjoying the fresh air. The sun is shining, and the birds are singing. It's a perfect day to get some exercise and clear your mind.\nDo you have any plans for the day? I was thinking of trying out a new recipe in the kitchen. I've been wanting to try out some new flavors and ingredients.\nI'm so glad you reminded me to get outside and enjoy the weather. You're always so full"], 'num_tokens': 175}]


In [7]:
from parallelformers import parallelize

In [8]:
parallelize(model, num_gpus=2, fp16=True, verbose='detail')

Traceback (most recent call last):
  File "/opt/kernel/lib/python3.11/site-packages/parallelformers/parallelize.py", line 287, in parallelize
    p_mutex.wait()
  File "/opt/kernel/lib/python3.11/multiprocessing/synchronize.py", line 356, in wait
    self._cond.wait(timeout)
  File "/opt/kernel/lib/python3.11/multiprocessing/synchronize.py", line 268, in wait
    return self._wait_semaphore.acquire(True, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [1]:
import os
from accelerate.utils import write_basic_config

write_basic_config()  # Write a config file

PosixPath('/home/bqnt-user/.cache/huggingface/accelerate/default_config.yaml')

In [2]:
from accelerate import notebook_launcher

https://www.kaggle.com/code/blacksamorez/tensor-parallel-int4-llm/

Tensor_parallel

In [5]:
%pip install tensor_parallel

DEPRECATION: Loading egg at /opt/kernel/lib/python3.11/site-packages/mamba_cell_magic-0.15.0-py3.7.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensor_parallel as tp
import transformers

In [7]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

In [9]:
model = tp.TensorParallelPreTrainedModel( # <- tensor parallelism starts here
    model,
    device_ids=["cuda:0", "cuda:1"],
)

TensorParallelPreTrainedModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [17]:
def run_model(prompt, tokenizer, model):
    #tokens = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([prompt], padding=True, truncation=True, return_tensors='pt')
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000)
    parsed_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    return tokenizer.batch_decode(parsed_ids, skip_special_tokens=True)[0]

In [18]:
run_model(prompt, model, tokenizer)

TypeError: Caught TypeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/kernel/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/kernel/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/kernel/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/kernel/lib/python3.11/site-packages/transformers/models/llama/modeling_llama.py", line 1163, in forward
    outputs = self.model(
              ^^^^^^^^^^^
  File "/opt/kernel/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           

In [15]:
print(tokenizer.decode(model.generate(
    **inputs,
    pad_token_id=tokenizer.eos_token_id,
    max_length=20,
)[0], skip_special_tokens=True))

NameError: name 'inputs' is not defined

In [16]:
pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16}
    )

Device set to use cuda:0
The model 'TensorParallelPreTrainedModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', '

In [ ]:
model.generate()